In [ ]:
!pip3 install datasets

In [ ]:
import os
import torch
import pandas as pd

from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from datasets import Dataset

In [ ]:
%pwd

'/content'

### Loading the dataset

In [ ]:
df = pd.read_csv('../data/dataset.csv')
label_encoder = LabelEncoder()
df['label'] = label_encoder.fit_transform(df['category'])
df.label.value_counts()

,count
label,
0,132
2,88
1,55


In [ ]:
len(df.label.value_counts())

3

### Splitting of the data and converting to huggingface format

In [ ]:
train_texts, val_texts, train_labels, val_labels = train_test_split(
    df['content'].tolist(), df['label'].tolist(), test_size=0.2, random_state=42
)

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Creation of the encodings using BERT Tokenizer
train_encodings = tokenizer(train_texts , truncation=True, padding=True, max_length=512)
val_encodings   = tokenizer(val_texts   , truncation=True, padding=True, max_length=512)

# converting it into huggingface format
train_dataset = Dataset.from_dict({'input_ids': train_encodings['input_ids'],
                                   'attention_mask': train_encodings['attention_mask'],
                                   'labels': train_labels})

val_dataset = Dataset.from_dict({'input_ids': val_encodings['input_ids'],
                                 'attention_mask': val_encodings['attention_mask'],
                                 'labels': val_labels})

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


### Initilising the trainer

In [ ]:
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=len(df.label.value_counts()))

training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy="epoch",
    per_device_train_batch_size=24,
    per_device_eval_batch_size=24,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


### Finetuning the model

In [ ]:
trainer.train()

trainer.save_model('../finetuned_model/fine-tuned-bert')

Epoch,Training Loss,Validation Loss
1,0.956600,0.707157
2,0.557100,0.412349
3,0.365500,0.350497


### Loading the finetuned model

In [ ]:
checkpoint_dir = "./checkpoint-30"
model = BertForSequenceClassification.from_pretrained(checkpoint_dir)
tokenizer = BertTokenizer.from_pretrained(checkpoint_dir)
model.eval()

### Trying the model on few samples

In [ ]:
def predict_category(texts):
    encodings = tokenizer(texts, truncation=True, padding=True, max_length=512, return_tensors="pt")
    outputs = model(**encodings)
    preds = torch.argmax(outputs.logits, dim=1).numpy()
    return label_encoder.inverse_transform(preds)


In [ ]:
test_texts = ["Career Counseling Appointment: How can I schedule an appointment with a career counselor? I need guidance on my major selection.",
              "Industry Placement Year: We're interested in offering year-long industry placements. How can we set this up with your university?",
              "Conference Organization Committee: Would you be interested in joining the organizing committee for next year's conference on climate change?"
             ]
print(predict_category(test_texts))


['student' 'corporate' 'researcher']
